## **CyberBullying Detection based on NLP with BERT**
###**41043 Natural Language Processing Natural Language Processing Algorithms**

**XiaoTongf Lu - 14163950**

### **1. Install Package**

Force OS input command in UTF-8

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Install NeatText and transformer

In [ ]:
!pip install neattext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.1 MB/s eta 0:00:00


Import and Download stopwords

In [ ]:
import spacy
import re
import nltk
import string
import sklearn
import neattext as nt
import neattext.functions as nfx
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from keras import layers
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping
from keras.initializers import TruncatedNormal
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import AutoTokenizer, TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### **2.Read In File**

Moun the disk

In [ ]:
# mount disk
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


locate file location

In [ ]:
!ls /content/gdrive/MyDrive/NLP

'Assignment 3.ipynb'	   'Untitled document.gdoc'
 cyberbullying_tweets.csv  'Untitled presentation.gslides'


In [ ]:
%cd /content/gdrive/

/content/gdrive


Read in file

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/NLP/cyberbullying_tweets.csv')

### **3.File Inspection**

read file head and general data

In [ ]:
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
data.groupby('cyberbullying_type').describe()

tweet_text         \
                         count unique   
cyberbullying_type                      
age                       7992   7992   
ethnicity                 7961   7959   
gender                    7973   7948   
not_cyberbullying         7945   7937   
other_cyberbullying       7823   7823   
religion                  7998   7997   

                                                                             
                                                                   top freq  
cyberbullying_type                                                           
age                  Here at home. Neighbors pick on my family and ...    1  
ethnicity            Racism won't stop as long as u stil select ur ...    2  
gender               We proved that we're not just pretty faces. Oh...    2  
not_cyberbullying    Strategicscoring should be classed as cheating...    2  
other_cyberbullying  @ikralla fyi, it looks like I was caught by it...    1  
religion             A Pakistani court has sentenced 86 members of ...    2

In [ ]:
data['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [ ]:
data['cyberbullying_type'].nunique()

6

In [ ]:
data['cyberbullying_type'].count()

47692

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [ ]:
data.shape

(47692, 2)

### **4.Dataset Modification**

In [ ]:
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
data.tail()

,tweet_text,cyberbullying_type
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity
47691,Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...,ethnicity


In [ ]:
data.isnull().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

Remove the instance that are duplicated

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

(47656, 2)

Split train test data

In [ ]:
data_train, data_test = train_test_split(data, test_size = 0.3, random_state = 42, shuffle = True, stratify = data.cyberbullying_type)

**DataClean Up** 

---


remove #

In [ ]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_hashtags)
data_train.head()

,tweet_text,cyberbullying_type
37246,when cis ppl spread hate & misinfo about trans...,age
12581,@Thats_So_Lauren they like making over the top...,gender
29310,"So, just to save you all some time, BSD is my ...",other_cyberbullying
44701,Fuck I fucking hate dumb Hawaiians with nigger...,ethnicity
30210,RT @grbradbury: @davidlipson @Nick_Xenophon ca...,other_cyberbullying


remove userhandles

In [ ]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_userhandles)

In [ ]:
data_train.tail()

,tweet_text,cyberbullying_type
44713,Ur first one that is so apt!,ethnicity
31973,Art was something I turned to when I was raped...,age
46332,I disagree with your reading of it. Why would ...,ethnicity
14085,she was hating on jlaw for things she did when...,gender
43904,first impression: very blunt and straightforwa...,ethnicity


Remove extra spaces

In [ ]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_multiple_spaces)
data_train.head()

,tweet_text,cyberbullying_type
37246,when cis ppl spread hate & misinfo about trans...,age
12581,they like making over the top gay jokes invol...,gender
29310,"So, just to save you all some time, BSD is my ...",other_cyberbullying
44701,Fuck I fucking hate dumb Hawaiians with nigger...,ethnicity
30210,RT can you please call him an evil capitalist ...,other_cyberbullying


Remove stop words

In [ ]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_stopwords)
data_train.head()

,tweet_text,cyberbullying_type
37246,cis ppl spread hate & misinfo trans ppl though...,age
12581,like making gay jokes involving rape...not fun...,gender
29310,"So, save time, BSD family. negative response G...",other_cyberbullying
44701,Fuck fucking hate dumb Hawaiians nigger lips!,ethnicity
30210,RT evil capitalist sellout cunt https://t.co/N...,other_cyberbullying


Remove url

In [ ]:
data_train['tweet_text'] = data_train['tweet_text'].apply(nfx.remove_urls)
data_train.head()

,tweet_text,cyberbullying_type
37246,cis ppl spread hate & misinfo trans ppl though...,age
12581,like making gay jokes involving rape...not fun...,gender
29310,"So, save time, BSD family. negative response G...",other_cyberbullying
44701,Fuck fucking hate dumb Hawaiians nigger lips!,ethnicity
30210,RT evil capitalist sellout cunt,other_cyberbullying


### **5. Encode Label**

declare encoder

In [ ]:
label_enc = LabelEncoder() 

Encode label into number

In [ ]:
data_train['cyberbullying_type'] = label_enc.fit_transform(data_train['cyberbullying_type'])
data_train.head()

,tweet_text,cyberbullying_type
37246,cis ppl spread hate & misinfo trans ppl though...,0
12581,like making gay jokes involving rape...not fun...,2
29310,"So, save time, BSD family. negative response G...",4
44701,Fuck fucking hate dumb Hawaiians nigger lips!,1
30210,RT evil capitalist sellout cunt,4


Read the numbers in relationship with original names

In [ ]:
for index, label in enumerate(label_enc.classes_):
    print(f'Encoded Label {index} corresponds to original label "{label}"')

Encoded Label 0 corresponds to original label "age"
Encoded Label 1 corresponds to original label "ethnicity"
Encoded Label 2 corresponds to original label "gender"
Encoded Label 3 corresponds to original label "not_cyberbullying"
Encoded Label 4 corresponds to original label "other_cyberbullying"
Encoded Label 5 corresponds to original label "religion"


In [ ]:
data_test['cyberbullying_type'] = label_enc.transform(data_test['cyberbullying_type'])

In [ ]:
data_test['cyberbullying_type'].unique()

array([0, 4, 5, 2, 1, 3])

In [ ]:
data_train.dtypes

tweet_text            object
cyberbullying_type     int64
dtype: object

In [ ]:
data_train['cyberbullying_type'].unique()

array([0, 2, 4, 1, 3, 5])

In [ ]:
data_train['cyberbullying_type'].value_counts()

5    5598
0    5594
1    5571
2    5564
3    5556
4    5476
Name: cyberbullying_type, dtype: int64

### **6.Tokenize train input**

Tokenlize train

In [ ]:
x_train = tokenizer(
#     text = x_train.tolist(),
    text = data_train['tweet_text'].tolist(),
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [ ]:
x_train['input_ids']

<tf.Tensor: shape=(33359, 100), dtype=int32, numpy=
array([[  101,   172,  1548, ...,     0,     0,     0],
       [  101,  1176,  1543, ...,     0,     0,     0],
       [  101,  1573,   117, ...,     0,     0,     0],
       ...,
       [  101, 23423,  3455, ...,     0,     0,     0],
       [  101, 26766,   179, ...,     0,     0,     0],
       [  101,  8351,   131, ...,     0,     0,     0]], dtype=int32)>

Tokenlize test

In [ ]:
x_test = tokenizer(
    text = data_test['tweet_text'].tolist(), 
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

### **7.Constrcut Model**

In [ ]:
max_len = 100


input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
input_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')


embeddings = bert(input_ids, attention_mask=input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)

y = Dense(6, activation='sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
# Accordingly Bert model requires a specific learning rate

optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
)

loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

### **8.Train Model**

In [ ]:
bert_train = model.fit(
    x={'input_ids':x_train['input_ids'], 'attention_mask':x_train['attention_mask']},
    y=to_categorical(data_train.cyberbullying_type),
    validation_data=(
        {'input_ids':x_test['input_ids'], 'attention_mask':x_test['attention_mask']},to_categorical(data_test.cyberbullying_type)
    ),
    epochs=1,
    batch_size=36
)

/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


927/927 [==============================] - 295s 292ms/step - loss: 0.5022 - balanced_accuracy: 0.7985 - val_loss: 0.4071 - val_balanced_accuracy: 0.8352


### **9.Evaluation**

In [ ]:
pred_raw = model.predict({'input_ids':x_test['input_ids'], 'attention_mask':x_test['attention_mask']})

447/447 [==============================] - 40s 83ms/step


In [ ]:
pred_raw[0]

array([0.9986474 , 0.00247782, 0.02847658, 0.8229654 , 0.45240712,
       0.00356158], dtype=float32)

In [ ]:
y_pred = np.argmax(pred_raw, axis=1)

In [ ]:
data_test.cyberbullying_type 

31785    0
30876    4
17444    5
9823     2
45433    1
        ..
25842    4
36598    0
31565    4
1359     3
28625    4
Name: cyberbullying_type, Length: 14297, dtype: int64

In [ ]:
print(classification_report(data_test.cyberbullying_type, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2398
           1       0.95      0.97      0.96      2388
           2       0.89      0.84      0.87      2384
           3       0.65      0.52      0.58      2381
           4       0.62      0.72      0.66      2347
           5       0.93      0.97      0.95      2399

    accuracy                           0.84     14297
   macro avg       0.83      0.83      0.83     14297
weighted avg       0.83      0.84      0.83     14297

